<a href="https://colab.research.google.com/github/HamidrezaGh/Physical_Distancing_Detector/blob/master/social_dist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import imutils
import time
import argparse
import numpy as np
from scipy.spatial import distance as dist

In [ ]:
# Application Constant configurations

# Minimum probability to filter weak detections
MIN_CONFIDENCE = 0.3
THERESHOLD = 0.3

# Minimum Social Distancing which is safe according to the Healthcare Professionals
MIN_DISTANCE = 30  

# YOLO directory path
MODEL_PATH = "yolo-coco"

In [ ]:
def detect_object(frame, network, output_layer, cid):
    
    
    # Get the current frame dmension
    (H, W) = frame.shape[:2]
    
    # initialize variables
    confidences = []
    bounding_boxes = []
    centers = []
    results = []

    # Getting blob from the current frame
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    network.setInput(blob)
    layerOutputs = network.forward(output_layer)

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the confidence and class ID
            scores = detection[5:]
            confidence = scores[cid]
            classID = np.argmax(scores)

            # filter only person detection with confidence more that MIN_CONFIDENCE
            if classID == cid and confidence > MIN_CONFIDENCE:
                # scale the bounding box 
                box = detection[0:4] * np.array([W, H, W, H])
                (center_X, center_Y, width, height) = box.astype("int")

                # Get the top left corner of the box
                x = int(center_X - (width / 2))
                y = int(center_Y - (height / 2))

                # append to the lists
                confidences.append(float(confidence))
                centers.append((center_X, center_Y))
                bounding_boxes.append([x, y, int(width), int(height)])
                

    # apply non-maxima suppression
    idxs = cv2.dnn.NMSBoxes(bounding_boxes, confidences, MIN_CONFIDENCE, THERESHOLD)

    # check the availability of at least one detection
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (bounding_boxes[i][0], bounding_boxes[i][1])
            (w, h) = (bounding_boxes[i][2], bounding_boxes[i][3])

            # Update the result list and return
            r = (confidences[i], (x, y, x + w, y + h), centers[i])
            results.append(r)

    return results

In [ ]:
def physical_distancing_detector():
    
    # Input and Output videos
    video = {

        'input_video': 'pedestrians.mp4',
        'output_video': 'pedestrians_output.mp4',
        "frame_count": 0,
        "processing_time": 0
    }
    
    start = time.time()
       
    # Read the Video File
    input_video = cv2.VideoCapture(video["input_video"] if video["input_video"] else 0)
    writer = None

    # load labels from YOLO model
    lables = []
    with open(MODEL_PATH + '/coco.names') as file:
        lables = [line.strip() for line in file]
    
    network = cv2.dnn.readNetFromDarknet(MODEL_PATH + '/yolov3.cfg', MODEL_PATH + '/yolov3.weights')
  
    # Get the output layers that require from YOLO
    all_layer = network.getLayerNames()
    output_layer = [all_layer[i[0] - 1] for i in network.getUnconnectedOutLayers()]

    
    print('start processing the video frame by frame ...')
    
    while True:
        
        video['frame_count'] = video['frame_count'] + 1

        # set of people that violate social distancing role 
        violateSet = set()
        
        # Read the next frame
        (exist, currentFrame) = input_video.read()

        # Check the availability frame, if not (end of the video) the while loop finish
        if not exist:
            break

        # resize the frame and then detect objects (people) in it
        currentFrame = imutils.resize(currentFrame, width=500)
        results = detect_object(currentFrame, network, output_layer, lables.index("person"))
        
        if len(results) >= 2:
            # Get All centers
            centers = np.array([r[2] for r in results])
            eucDist = dist.cdist(centers, centers, metric="euclidean")
            
            for i in range(0, eucDist.shape[0]):
                for j in range(i + 1, eucDist.shape[1]):
                    if eucDist[i, j] < MIN_DISTANCE:
                        violateSet.add(i)
                        violateSet.add(j)

        # loop over the results to get centroid coordiantes and initialize color
        for (i, (prob, box, center)) in enumerate(results):
            (center_X, center_Y) = center
            (start_X, start_Y, end_X, end_Y) = box
            color = (0, 255, 0)
            
            # If i in the violetSet update the color
            if i in violateSet:
                color = (0, 0, 255)

            cv2.circle(currentFrame, (center_X, center_Y), 5, color, 1)
            cv2.rectangle(currentFrame, (start_X, start_Y), (end_X, end_Y), color, 2)
            

        # Total number of violation cases in the current frame
        text = "Violations Count: " + format(len(violateSet))
        cv2.putText(currentFrame, text, (10, currentFrame.shape[0] - 25),
            cv2.FONT_HERSHEY_COMPLEX, 0.9, (36 ,36 ,255), 3)


        # Generate output video
        if video["output_video"] != "" and writer is None:
            # initialize our video writer
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")
            writer = cv2.VideoWriter(video["output_video"], fourcc, 30,
                                     (currentFrame.shape[1], currentFrame.shape[0]), True)

        if writer is not None:
            writer.write(currentFrame)
    
    end = time.time()

    
    video['processing_time'] +=  end - start 
    print('Processing is end and the output file is: ' + video["output_video"])
    print()
    print('Total number of frames', video['frame_count'])
    print('Total processing duration {:.5f} seconds'.format( video['processing_time']))
    print('FPS:', round((video['frame_count'] / video['processing_time']), 1))

    # Releasing video reader and writer
    writer.release()
    input_video.release()
    

In [ ]:
physical_distancing_detector()

start processing the video frame by frame ...
Processing is end and the output file is: pedestrians_output.mp4

Total number of frames 532
Total processing duration 372.49065 seconds
FPS: 1.4
